In [4]:
import tensorflow as tf
import numpy as np
import sklearn as sk
import pandas as pd
from conv import input_data

In [9]:
#graph

#placeholder
x = tf.placeholder(shape=[None, 784], dtype=tf.float32)
y_ = tf.placeholder(shape=[None, 10], dtype=tf.float32)

#first layer
W_conv1 = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev=0.1))
b_conv1 = tf.constant(0.1, shape=[32])
x_image = tf.reshape(x, [-1, 28, 28, 1])
h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding="SAME") + b_conv1)
h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

#second layer
W_conv2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1))
b_conv2 = tf.constant(0.1, shape=[64])
h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], padding="SAME") + b_conv2)
h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

#ouput
W_fc1 = tf.Variable(tf.truncated_normal([7*7*64, 1024], stddev=0.1))
b_fc1 = tf.constant(0.1, shape=[1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
keep_prob = tf.placeholder(dtype=tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob=keep_prob)

W_fc2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1))
b_fc2 = tf.constant(0.1, shape=[10])
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

#predict
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32))

In [10]:
#input_data
minist = input_data.read_data_sets("MINIST_data", one_hot=True)

Extracting MINIST_data/train-images-idx3-ubyte.gz


Extracting MINIST_data/train-labels-idx1-ubyte.gz
Extracting MINIST_data/t10k-images-idx3-ubyte.gz
Extracting MINIST_data/t10k-labels-idx1-ubyte.gz


In [19]:
#train
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())

for i in range(301):
    batch = minist.train.next_batch(100)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_:batch[1], keep_prob:0.5})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

/Users/apple/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1714: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


step 0, training accuracy 0.1


step 100, training accuracy 0.91


step 200, training accuracy 0.81


step 300, training accuracy 0.92


In [20]:
#test
print("test accuracy %g"%accuracy.eval(feed_dict = {x: minist.test.images, y_: minist.test.labels, keep_prob: 1.0}))

test accuracy 0.9434
